In [100]:
import numpy as np
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [104]:
def show_images(image):
    """
    image - image to be shown
    """

    fig, axes = plt.subplots(2, 4, figsize=(10, 6))

    for i, ax in enumerate(axes.flatten()):
        ax.imshow(image)
        ax.axis('off') 

    plt.tight_layout()

    # Show the plot
    plt.show()

In [41]:
def ela(imagePath, scale=10, quality=90):
    """
    Performs Error Level Analysis (ELA) on an image.
    imagePath (str) - the path to the image file.
    scale (int) - the scale factor for resizing the image. Default is 10.

    Returns:
        elaImage (np.array) - the ELA image.
    """
    # Load the image
    image = cv2.imread(imagePath)

    # Resize the image
    resizedImage = cv2.resize(image, (scale, scale))

    # Convert the image to JPEG
    cv2.imwrite("temp.jpg", resizedImage, [cv2.IMWRITE_JPEG_QUALITY, quality])

    # Load the JPEG image and compute the ELA image
    elaImage = cv2.imread("temp.jpg")
    elaImage = cv2.absdiff(resizedImage, elaImage)
    elaImage = cv2.cvtColor(elaImage, cv2.COLOR_BGR2GRAY)

    return elaImage

#### RPA

In [43]:
def rpa(elaImage, threshold=5):
    """
    Residual Pixel Analysis (RPA) on an ELA image.

    Args:
        elaImage (np.array) - ELA image.

    Returns:
        tampered (bool) - True if the image is tampered, False otherwise.
    """
    # Calculate the standard deviation of the ELA image
    stddev = np.std(elaImage)

    # If the standard deviation is greater than the threshold, the image is tampered
    tampered = stddev > threshold

    return tampered

In [112]:
def detectFraud(imagePath, scale, quality, threshold):
    """
    Detects identity card fraud using ELA and RPA.

    Args:
        imagePath (str) -  path to the image file.

    Returns:
        result (str) - the result of the fraud detection, either "Genuine" or "Tampered".
    """
    
    # Perform ELA on the image
    elaImage = ela(imagePath, scale, quality)
    # show_images(elaImage)

    # Perform RPA on the ELA image
    tampered = rpa(elaImage, threshold)

    # Determine the result
    if tampered:
        result = 1
    else:
        result = 0

    return result

In [47]:
# Define source paths
authentic = 'data/casia/au/'
tampered = 'data/casia/tp/'

In [122]:
# Mix photos
X = list()
Y = list()
for files in os.listdir(authentic):
    X.append(authentic+files)
    Y.append('Au')
for files in os.listdir(tampered):
    X.append(tampered+files)
    Y.append('Tp')
    
le = preprocessing.LabelEncoder()
y = le.fit_transform(Y)

In [124]:
xTrain, xTest, yTrain, yTest = train_test_split(X,y,train_size = 0.8, random_state=40,shuffle=True, stratify=y)

In [150]:
tp,fp,fn = 0,0,0
for scale in [10,20,30]:
    for quality in [90]:
        for threshold in [3,5,7]:
            tp,fp,fn, score = 0,0,0,0
            for i in xTrain[:10]:
                cnt = detectFraud(i, scale, quality, threshold)
                if cnt == 0 and yTrain[xTrain.index(i)] == 0:
                    tp += 1 
                elif cnt == 0 and yTrain[xTrain.index(i)] == 1:
                    fp += 1
                elif cnt == 1 and yTrain[xTrain.index(i)] == 0:
                    fn += 1
                        
                if cnt == yTrain[xTrain.index(i)]:
                    score += 1
            
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            F1 = 2*precision*recall/(precision+recall)
            print(F1,score/10, scale, quality, threshold)

0.5 0.4 10 90 3
0.9411764705882353 0.9 10 90 5
1.0 1.0 10 90 7
0.8 0.7 20 90 3
0.9411764705882353 0.9 20 90 5
0.9473684210526316 0.9 20 90 7
0.9411764705882353 0.9 30 90 3
0.9411764705882353 0.9 30 90 5
1.0 1.0 30 90 7


In [148]:
scale, quality, threshold = 10,90,7
tp,fp,fn, score = 0,0,0,0
for i in xTest[:20]:
    try:
        cnt = detectFraud(i, scale, quality, threshold)
        if cnt == 0 and yTest[xTest.index(i)] == 0:
            tp += 1 
        elif cnt == 0 and yTest[xTest.index(i)] == 1:
            fp += 1
        elif cnt == 1 and yTest[xTest.index(i)] == 0:
            fn += 1

        if cnt == yTest[xTest.index(i)]:
            score += 1
    except:
        pass
precision = tp/(tp+fp)
recall = tp/(tp+fn)
F1 = 2*precision*recall/(precision+recall)
print(F1,score/20, scale, quality, threshold)

0.72 0.6 10 90 7
